# Helper functions for salt-dilution notebook

In [1]:
using LsqFit # a package for least square fitting
"""
    fit_calibration(bucketsize, solution, calis...)

Fits a line of best fit through the calibration data going through the origin.  
Returns the function of this line: `f(cond-cond_at_0) -> concentration`.

Also prints the parameters values +/- 95% confidence intervals.

Uses the package LsqFit: https://github.com/JuliaOpt/LsqFit.jl
"""
function fit_calibration(bucketsize, solution, calis...)
    # subtract readout at zero concentration
    for c in calis
        @assert c[1,1]==0 "First row needs to be zero reading!"
        c[:,2] .= c[:,2].-c[1,2]
    end
    # concatenate all calibrations
    cali = vcat(calis...)
    conc = ml_to_concentration(cali[:,1], solution, bucketsize)
    delta_readout = cali[:,2]
    # Fit line using https://github.com/JuliaOpt/LsqFit.jl
    fn(delta_readout, p) = p[1]*delta_readout # p[1]==a, p[2]==b
    para_weights = [0.5] # equal weights to parameters
    fit = curve_fit(fn, delta_readout, conc, para_weights)
    errors = margin_error(fit, 1-0.95)
    println("""
    Estimated linear fit: f(delta_cond) = a*conc with
     a = $(round(fit.param[1],sigdigits=3))±$(round(errors[1],sigdigits=3))
    """)
    return (delta_readout) -> fn(delta_readout, fit.param)
end;

┌ Info: Precompiling LsqFit [2fda8390-95c7-5789-9bda-21331edee243]
└ @ Base loading.jl:1242


In [8]:
using Dates
using DelimitedFiles: readdlm # Date time handling; CSV file handling

# These will read files containing time series of a combination of the following sensors:
# conductivity, pressure, temperature
# The convention is to return a dict with keys as appropriate:
# :t [date-time stamp], :cond [μS/cm], :temp [C], :press [m H2O]
#
# (Note, the funny ":" in front of the name makes a special kind of string, a so-called Symbol)

"""
    read_WTW(filename)

This function reads a file from the WTW conductivity sensor and
returns:

Dict with keys: :t [date-time stamp], :cond [μS/cm], :temp [C]

Note, that the input file usually contains several traces.  Split them up with 
`split_conductivity_data`.
"""
function read_WTW(filename)
    if !isfile(filename)
        error("Filename $filename is not a file!")
    end
    if !startswith(splitdir(filename)[2], "AD")
        warn("Read in a file starting with `AD`.  (The `AC` files use a comma for the decimal point.")
    end
    d, head = readdlm(filename, ';', header=true)
    out = Dict{Symbol,Any}() # key has the be Symbol, value can be anything
    # time 12.08.2016 13:36:58
    fmt = "d.m.y H:M:S"
    out[:t] = [DateTime(dd, fmt) for dd in d[:,4]]
    # conductivity
    out[:cond] = d[:,5]
    units = d[:,6]
    @assert all(units.=="\xb5S/cm") "Units not in μS/cm!"
    # temp
    out[:temp] = d[:,8]
    
    # purge any records which are simultaneous (not sure why this happens with the WTW)
    purge = findall(diff(out[:t]).==Second(0))
    for p in reverse(purge)
        deleteat!(out[:t], p)
        deleteat!(out[:cond],p)
        deleteat!(out[:temp], p)
    end

    return out
end

    using DelimitedFiles, Dates

const g = 9.81
const rhow = 1000.0

"""
         read_Keller(filename;
                     presshead="P1",
                     condhead="ConRaw",
                     temphead="TOB1",
                     skipstart=8,
                     )

Reads a Keller pressure/CTD sensor.  However, you probably want to use
- `read_Keller_DCX22_CTD`, 
- `read_Keller_DCX22` and 
- `read_Keller_DC22` 

Returns a dict with keys as appropriate:
- :t [date-time stamp]
- :cond [μS/cm]
- :temp [C]
- :press [m H2O]
"""
function read_Keller(filename;
                     presshead="P1",
                     condhead="ConRaw",
                     temphead="TOB1",
                     skipstart=8,
                     )
    d,h = readdlm(filename, ';', skipstart=skipstart, header=true)
    h = h[:] # h is a 1x2 matrix, change to a vector

    out = Dict{Symbol,Any}()
    # find date-time rows
    id, it = findfirst(h.=="Date"), findfirst(h.=="Time")
    # time 12.08.2016 13:36:58
    fmtd, fmtt = "d/m/y", "H:M:S"
    out[:t] = [Date(dd, fmtd) + Time(tt, fmtt) for (dd,tt) in zip(d[:,id], d[:,it])]

    #
    for (head, key) in [(presshead, :press),
                        (condhead, :cond),
                        (temphead, :temp)]
        i = findfirst(h.==head) # see if there is one
        tmp = Float64[]
        if i!=nothing
            out[key] = [s=="" ? missing :
                        s isa AbstractString ? parse(Float64, replace(s, ","=>".")) : Float64(s) for s in d[:,i]]
            # convert mS/cm to μS/cm
            if key==:cond
                out[:cond] = out[:cond].*1000
            end
            # convert pressure from mbar to m H2O
            if key==:press
                out[:press] = out[:press]*1e2 /g/rhow
            end
        end
    end

    # check lengths and remove all "missing"
    l = length(out[:t])
    topurge = []
    for v in values(out)
        @assert length(v)==l
        append!(topurge, findall(v.===missing))
    end
    topurge = sort(unique(topurge))
    for (k,v) in out
        deleteat!(v, topurge)
        if k!=:t
            out[k] = Float64.(v) # make the vector an
        end
    end
    return out
end

read_Keller_DCX22_CTD(filename) = read_Keller(filename)
read_Keller_DCX22(filename) = read_Keller(filename, error("Not implement yet"))
read_Keller_DC22(filename) = read_Keller(filename,
                                         presshead="p1/mbar",
                                         temphead="t1/\xb0C",
                                         skipstart=24
                                         )

read_Keller_DC22 (generic function with 1 method)

In [19]:
"""
    split_conductivity_data_timegap(outdict, timegap=5.0, mincond=0.5, maxtemp=2)

Splits up the time series returned by `read_conductivity_data` into individual
traces, mostly by using that there is a big time-gap in the record.

First the datapoints which have `outdict[:cond]<mincond` or `outdict[:temp]>maxtemp` are deleted.
Then, the split is done where the time-gap is bigger than `timegap` seconds.

The output is a list of traces:

`traces = [trace1, trace2, etc]`

where `trace1` contains the time series with `:t` in seconds after recoding started, `:tstart` is 
the date-time of the start of the recording.  You should also add the injection `:mass` to each trace 
manually.

Thus to access the times of the second trace do `traces[2][:t]`, and to get the corresponding
conductivity readout `traces[2][:cond]`.
"""
function split_conductivity_data_timegap(out; timegap=5.0, mincond=0.0, maxtemp=2, minpeak=6, minduration=60)
    # delete where cond<mincond and temp>maxtemp
    out = deepcopy(out)
    tokeep = (out[:cond].>mincond) .& (out[:temp].<maxtemp)
    for (k,v) in out
        out[k] = v[tokeep]
    end

    t = out[:t]
    cond = out[:cond]   

    # split it up according to time gaps
    splits_at = findall(diff(t).>Second(timegap))
    push!(splits_at, length(t))
    traces = []
    i = 1
    for sp in splits_at
        tt = convert(Vector{Float64}, Dates.value.(t[i:sp]-t[i])/1000)
        dt = t[i+1]-t[i]
        if !all(diff(t[i:sp]).==dt) 
            #@warn "Logging intervals are not evenly spaced!"
        end
        tmp = Dict{Symbol,Any}()
        tmp[:t] = tt
        if tt[end]-tt[1]<minduration
            continue
        end
        tmp[:tstart] = t[i]
        tmp[:cond] = cond[i:sp]
        if maximum(tmp[:cond])<minpeak
            continue
        end
        if haskey(out, :temp)
            tmp[:temp] = out[:temp][i:sp]
        end
        if haskey(out, :press)
            tmp[:press] = out[:press][i:sp]
        end
        push!(traces, tmp)
        i = sp+1
    end
    
    return traces
end

"""
     split_conductivity_data(out;
                             slice_cond=2.5,  # peaks are selected where cond is above this value
                             slice_padding_start=20, # add a padding of in front of detected peak
                             slice_padding_end=3*60, # add a padding of in back of detected peak
                             mincond=0.5, maxtemp=3, # also drop points which have conductivity below mincond and above maxtemp
                             smooth_window=10) # smoothing window used in peak detection

Splits up the time series returned by `read_conductivity_data` into individual
traces, mostly by heuristics around where peaks are.  A bit hit and miss.

The output is a list of traces:

`traces = [trace1, trace2, etc]`

where `trace1` contains the time series with `:t` in seconds after recoding started, `:tstart` is
the date-time of the start of the recording.  You should also add the injection `:mass` to each trace
manually.

Thus to access the times of the second trace do `traces[2][:t]`, and to get the corresponding
conductivity readout `traces[2][:cond]`.
"""
function split_conductivity_data(out;
                                 mincond=0.5, maxtemp=3,
                                 slice_cond=2.5,
                                 slice_padding_start=20,
                                 slice_padding_end=3*60,
                                 smooth_window=10)
    out = deepcopy(out)

    # smooth cond with running mean
    cond = out[:cond]
    t = out[:t]
    condsmooth = zeros(length(cond))
    for i=1:length(t)
        ## running mean
        tmp = 0.0
        n = 0
        for j=max(1,i-smooth_window):min(i+smooth_window, length(t))
            tmp += cond[j]
            n += 1
        end
        condsmooth[i] = tmp/n

        # ## median
        # condsmooth[i] = median(cond[max(1,i-smooth_window):min(i+smooth_window, length(t))])
    end

    # locate peaks with slice_cond and cut, also cut if time-gap bigger than timegap
    inds = []

    tokeep = condsmooth.>slice_cond
    # add +/- slice_inds
    s = 1
    while true
        s = findnext(tokeep, s)
        if s==nothing
            break
        end
        e = findnext(.!tokeep, s)
        push!(inds, max(1, s-slice_padding_start):min(e+slice_padding_end, length(tokeep)))
        s = e+1
        if s>length(tokeep)
            break
        end
    end

    traces = []
    i = 1
    for ind in inds
        tt = convert(Vector{Float64}, Dates.value.(t[ind]-t[ind[1]])/1000)
        tmp = Dict{Symbol,Any}()

        # delete where cond<mincond and temp>maxtemp
        tokeep = (out[:cond][ind].>mincond) .& (out[:temp][ind].<maxtemp)

        tmp[:t] = tt[tokeep]
        tmp[:tstart] = t[ind][tokeep][1]
        tmp[:cond] = cond[ind][tokeep]
        tmp[:condsmooth] = condsmooth[ind][tokeep]
        push!(traces, tmp)
    end
    return traces
end
;